In [1]:
%cd /home/mbudisic/Documents/PsTuts-RAG
import os
from getpass import getpass
from dotenv import load_dotenv
from pstuts_rag.configuration import Configuration
import asyncio

import nest_asyncio
nest_asyncio.apply()


/home/mbudisic/Documents/PsTuts-RAG


In [2]:
import pstuts_rag

In [3]:
import logging
logging.basicConfig(
    level=Configuration().eva_log_level,
    format='%(asctime)s - %(levelname)s - %(funcName)s - %(message)s'
)

In [4]:
%load_ext autoreload
%autoreload 2


In [ ]:
load_dotenv()


True

In [6]:
from pstuts_rag.rag_for_transcripts import *

2025-05-31 00:25:16,293 - INFO - <module> - Loaded .env file


In [7]:
Configuration().print(logging.info)

2025-05-31 00:25:18,096 - INFO - print - Configuration parameters:
2025-05-31 00:25:18,097 - INFO - print -   eva_workflow_name: EVA_workflow
2025-05-31 00:25:18,099 - INFO - print -   eva_log_level: INFO
2025-05-31 00:25:18,100 - INFO - print -   transcript_glob: ./data/test.json
2025-05-31 00:25:18,101 - INFO - print -   embedding_model: mxbai-embed-large
2025-05-31 00:25:18,102 - INFO - print -   eva_strip_think: True
2025-05-31 00:25:18,103 - INFO - print -   embedding_api: ModelAPI.OLLAMA
2025-05-31 00:25:18,104 - INFO - print -   llm_api: ModelAPI.OLLAMA


2025-05-31 00:25:18,104 - INFO - print -   max_research_loops: 2
2025-05-31 00:25:18,105 - INFO - print -   llm_tool_model: qwen3:4b
2025-05-31 00:25:18,106 - INFO - print -   n_context_docs: 3


In [8]:
datastore = DatastoreManager()
datastore.add_completion_callback(lambda _: logging.warning("Loading complete."))
asyncio.create_task(datastore.from_json_globs(Configuration().transcript_glob))

2025-05-31 00:25:19,818 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:19,844 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


<Task pending name='Task-1' coro=<DatastoreManager.from_json_globs() running at /home/mbudisic/Documents/PsTuts-RAG/pstuts_rag/pstuts_rag/datastore.py:105>>

In [9]:
await datastore.wait_for_loading()

2025-05-31 00:25:20,879 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:21,701 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:22,528 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:23,356 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:23,851 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:24,806 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:25,058 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:25,730 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 

True

In [10]:
chain = create_transcript_rag_chain(datastore)

In [11]:
response = chain.invoke({"question":"What is a layer?"})

2025-05-31 00:25:30,144 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-05-31 00:25:40,574 - INFO - _send_single_request - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [12]:
response.pretty_print()

================================== Ai Message ==================================



A **layer** in Photoshop is a separate component of an image, acting like a "flat pint of glass" stacked on top of others. Each layer holds its own content (like a photo or artwork) and can be hidden or shown using the **Eye icon** in the Layers panel.  

Key points from the video:  
- Layers are the building blocks of any image in Photoshop.  
- They allow you to edit parts of an image independently without affecting other parts.  
- Transparent areas in a layer let you see through to layers below (e.g., a checkerboard pattern in the example).  
- The **Layers panel** lets you manage and select individual layers for editing.  

Timestamp: **00:00:00–00:00:23** (first explanation).  

This is covered in the "Understand layers" video. 🎨
**REFERENCES**
[
  {
    "title": "Understand layers",
    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c53

In [13]:
response.additional_kwargs["context"]

[Document(metadata={'video_id': 19172, 'title': 'Understand layers', 'desc': 'Learn what layers are and why they are so useful.', 'length': '00:04:44.75', 'group': 'data/test.json', 'source': 'https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4', 'speech_start_stop_times': [[0.47, 3.41], [3.81, 9.13], [9.309999, 15.01], [15.299999, 20.57], [20.88, 23.3], [23.83, 27.93], [29.38, 32.79], [32.96, 33.92], [34.43, 40.21], [41.91, 45.37], [45.88, 49.01], [49.54, 55.130001], [55.72, 58.49], [58.72, 62.14]], 'start': 0.47, 'stop': 62.14, '_id': 21, '_collection_name': 'a31ff6bb-4d35-4b52-8612-55df01f799fa'}, page_content="Layers are the building blocks of any image in Photoshop CC. So, it's important to understand, what layers are and why to use them - which we'll cover in this video. If you're following along, open this layered image from the downloadable practi

In [14]:
Configuration()


Configuration(eva_workflow_name='EVA_workflow', eva_log_level='INFO', transcript_glob='./data/test.json', embedding_model='mxbai-embed-large', eva_strip_think=True, embedding_api=<ModelAPI.OLLAMA: 'OLLAMA'>, llm_api=<ModelAPI.OLLAMA: 'OLLAMA'>, max_research_loops=2, llm_tool_model='qwen3:4b', n_context_docs=3)